In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [2]:
!pip install peft --quiet

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Remove nrows = ? for 

In [ ]:
# df = pd.read_csv('/kaggle/input/cleaned-news-dataset/Cleaned_news_dataset.csv', usecols=['summary', 'article'], nrows=1000)

In [4]:
df = pd.read_csv('/kaggle/input/cleaned-news-dataset/Cleaned_news_dataset.csv', usecols=['summary', 'article'])

In [5]:
df

,summary,article
0,তুরস্কের পশ্চিমাঞ্চলীয় ইজমির প্রদেশে সুইডেনের ...,তুরস্কের পশ্চিমাঞ্চলীয় ইজমির প্রদেশে সুইডেনের ...
1,সারাদেশে ডেঙ্গু পরিস্থিতি দিন দিন আরও ভয়াবহ রূ...,সারাদেশে ডেঙ্গু পরিস্থিতি দিন দিন আরও ভয়াবহ রূ...
2,শোকাবহ আগস্টের প্রথম দিনে সুনামগঞ্জে স্বেচ্ছায়...,শোকাবহ আগস্টের প্রথম দিনে সুনামগঞ্জে স্বেচ্ছায়...
3,পানিসম্পদ উপমন্ত্রী একেএম এনামুল হক শামীম বলেছ...,পানিসম্পদ উপমন্ত্রী একেএম এনামুল হক শামীম বলেছ...
4,বিএনপির ‌‘অগ্নিসন্ত্রাস ও নৈরাজ্য সৃষ্টির প্রত...,বিএনপির ‌‘অগ্নিসন্ত্রাস ও নৈরাজ্য সৃষ্টির প্রত...
...,...,...
37760,বঙ্গবন্ধু শেখ মুজিবুর রহমান এমন একটি নাম—যে না...,বঙ্গবন্ধু শেখ মুজিবুর রহমান এমন একটি নাম—যে না...
37761,"আমার ভাই মনে কষ্ট নিয়ে চলে গেছে। সে মনে করত, ত...","আমার ভাই মনে কষ্ট নিয়ে চলে গেছে। সে মনে করত, ত..."
37762,সারাজীবন আমি ভেবেছি সাদি আর শিবলী আমরা প্রায় স...,সারাজীবন আমি ভেবেছি সাদি আর শিবলী আমরা প্রায় স...
37763,দেশে শিশুশ্রম বেড়েছে। বাংলাদেশ পরিসংখ্যান ব্যু...,দেশে শিশুশ্রম বেড়েছে। বাংলাদেশ পরিসংখ্যান ব্যু...


In [6]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.30, shuffle=True)


In [7]:
from datasets import Dataset
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

# Model on Alpaca-orca dataset

In [8]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("BanglaLLM/BanglaLLama-3.2-1b-bangla-alpaca-orca-instruct-v0.0.1")
model = AutoModelForCausalLM.from_pretrained("BanglaLLM/BanglaLLama-3.2-1b-bangla-alpaca-orca-instruct-v0.0.1")

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,   # Task type
    inference_mode=False,           # Enable training mode
    r=16,                           # LoRA rank
    lora_alpha=32,                  # Scaling factor
    lora_dropout=0.1                # Dropout rate
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.to(device)


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/917 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(128256, 2048)
          (modules_to_save): ModuleDict(
            (default): Embedding(128256, 2048)
          )
        )
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): Parameter

# Model on culturax dataset

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("BanglaLLM/BanglaLLama-3.2-1b-unolp-culturax-base-v0.0.1")
model = AutoModelForCausalLM.from_pretrained("BanglaLLM/BanglaLLama-3.2-1b-unolp-culturax-base-v0.0.1")

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,   # Task type
    inference_mode=False,           # Enable training mode
    r=16,                           # LoRA rank
    lora_alpha=32,                  # Scaling factor
    lora_dropout=0.1                # Dropout rate
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.to(device)


## Tokenize dataset with your model tokenizer

In [9]:
# Tokenize the data
def preprocess_function(examples):
    inputs = tokenizer(
        examples["article"], max_length=256, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        examples["summary"], max_length=256, truncation=True, padding="max_length"
    )

    # Replace pad token ID with -100 in labels (ignored during loss computation)
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels["input_ids"]
    ]

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"],
    }


In [10]:
from transformers import DataCollatorForSeq2Seq




tokenized_train = ds_train.map(
    preprocess_function, batched=True, remove_columns=["summary", "article", "__index_level_0__"]
)
tokenized_test = ds_test.map(
    preprocess_function, batched=True, remove_columns=["summary", "article", "__index_level_0__"]
)



# Define a data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    return_tensors="pt"
)


Map:   0%|          | 0/26435 [00:00<?, ? examples/s]

Map:   0%|          | 0/11330 [00:00<?, ? examples/s]

# Debug stuff

In [11]:
print(tokenized_train[0])


{'input_ids': [128000, 11372, 116, 50228, 105, 60008, 11372, 243, 36278, 116, 11372, 106, 50228, 250, 11372, 243, 11372, 110, 53906, 107, 50228, 96, 36278, 106, 87648, 53906, 97, 53906, 108, 28025, 222, 36278, 241, 36278, 110, 50228, 110, 11372, 106, 87648, 62456, 73358, 11372, 117, 50228, 253, 12, 28025, 101, 36278, 228, 11372, 116, 87648, 60008, 73358, 36278, 116, 11372, 224, 11372, 116, 11372, 99, 36278, 116, 11372, 99, 11372, 116, 53906, 107, 36278, 101, 28025, 223, 73358, 28025, 223, 11372, 250, 53906, 250, 50228, 106, 50228, 101, 36278, 228, 11372, 117, 11372, 106, 60008, 11372, 99, 60008, 73358, 36278, 249, 60008, 11372, 110, 60008, 36278, 108, 50228, 243, 81278, 105, 28025, 223, 11372, 250, 53906, 250, 50228, 106, 50228, 101, 36278, 228, 11372, 117, 11372, 106, 60008, 11372, 99, 36278, 248, 50228, 248, 50228, 108, 36278, 116, 11372, 247, 53906, 245, 60008, 36278, 103, 53906, 108, 11372, 97, 81278, 99, 53906, 105, 87648, 53906, 99, 53906, 105, 81278, 97, 42412, 36278, 243, 73358

In [12]:
train_loader = DataLoader(tokenized_train, batch_size=2, collate_fn=data_collator)
batch = next(iter(train_loader))
print({key: val.shape for key, val in batch.items()})

{'input_ids': torch.Size([2, 256]), 'attention_mask': torch.Size([2, 256]), 'labels': torch.Size([2, 256])}


In [13]:
batch = {k: v.to(device) for k, v in batch.items()}
outputs = model(**batch)
print("Loss:", outputs.loss)
print("Logits shape:", outputs.logits.shape)


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Loss: tensor(1.0500, device='cuda:0', grad_fn=<NllLossBackward0>)
Logits shape: torch.Size([2, 256, 128256])


In [14]:
model.gradient_checkpointing_enable()

batch = next(iter(train_loader))
batch = {k: v.to(device) for k, v in batch.items()}
outputs = model(**batch)
print("Logits:", outputs.logits.shape)

# Compute loss and backpropagate
loss = outputs.loss
loss.backward()
print(torch.cuda.memory_allocated(device) / 1024**3, "GB used after backward pass")


Logits: torch.Size([2, 256, 128256])
7.015652656555176 GB used after backward pass


# ------------------------------------------------------------------

# Train

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"
print("ran")

ran


In [16]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",          # Save model here
    eval_strategy="epoch",    # Evaluate every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=False,     # Generate summaries during eval
    logging_dir='./logs',           # For logging
    logging_steps=100,
    save_strategy="no",
    report_to="none",
    fp16 = True,
    dataloader_num_workers=2,        # Optimize data loading
    gradient_accumulation_steps=4,   # Adjust if batch size is small
    
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,0.727800,0.722407
1,0.701400,0.704902
2,0.683200,0.700519


TrainOutput(global_step=4956, training_loss=0.7261932440781612, metrics={'train_runtime': 30293.0486, 'train_samples_per_second': 2.618, 'train_steps_per_second': 0.164, 'total_flos': 1.5305501109859123e+17, 'train_loss': 0.7261932440781612, 'epoch': 2.999546073536087})

## Save model?

In [20]:
# Save the model and tokenizer
trainer.save_model("/kaggle/working/saved_model")  # Save in Kaggle working directory
tokenizer.save_pretrained("/kaggle/working/saved_model")  # Save tokenizer files


('/kaggle/working/saved_model/tokenizer_config.json',
 '/kaggle/working/saved_model/special_tokens_map.json',
 '/kaggle/working/saved_model/tokenizer.json')

In [21]:
import shutil

# Compress the saved model directory
shutil.make_archive("/kaggle/working/saved_model", 'zip', "/kaggle/working/saved_model")


'/kaggle/working/saved_model.zip'

# Get ROUGE scores

In [17]:
pip install rouge_score evaluate --quiet

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    encoded_arg = tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
    preds, labels = eval_arg

    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text

    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring

    # (Note : Please change this code, when you perform on other languages except for Bengali)
    text_preds = [(p if p.endswith(("!", "!", "?", "?", "।")) else p + "।") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "!", "?", "?", "।")) else l + "।") for l in text_labels]
    sent_tokenizer_bn = RegexpTokenizer(u'[^!!??।]*[!!??।]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(l))) for l in text_labels]

    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
    )

## Dataloader takes data in random sequence randomly, so results may vary

In [19]:
from torch.utils.data import DataLoader

testing_dataloader = DataLoader(
    tokenized_test.with_format("torch"),
    collate_fn=data_collator,
    batch_size=1
)

for batch in testing_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=2,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_new_tokens=256
        )
    labels = batch["labels"]
    break

metrics_func([preds, labels])

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'rouge1': 0.9162210338680927,
 'rouge2': 0.9123434704830053,
 'rougeL': 0.9162210338680927,
 'rougeLsum': 0.9168141592920354}